In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),os.pardir))
from src.data.load_preprocess_data import load_raw_complaints_data
import altair as alt
import pandas as pd
import warnings
warnings.simplefilter("ignore", UserWarning)
data_path = os.path.join(os.pardir, "data", "raw", "complaints.csv")

complaints_df = load_raw_complaints_data(data_path)

In [2]:
complaints_df = complaints_df.query('not consumer_disputed.isnull()')
complaints_df['consumer_disputed'].replace(['Yes','No'],[1,0], inplace = True)
drop_features = ['date_received',
                 'zip_code',
                 'tags',
                 'date_sent_to_company',
                 'complaint_id']
complaints_df = complaints_df.drop(columns = drop_features).dropna()

In [3]:
complaints_df.head()
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(complaints_df,test_size=0.2, random_state=123)

In [4]:
train_df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,consumer_consent_provided,submitted_via,company_response_to_consumer,timely_response,consumer_disputed
2226480,Student loan,Non-federal student loan,Dealing with my lender or servicer,Received bad information about my loan,WF XXXX continues to report that ( XXXX ) acco...,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,Consent provided,Web,Closed with explanation,Yes,0
1134000,Debt collection,"Other (i.e. phone, health club, etc.)",Communication tactics,Used obscene/profane/abusive language,I did join the gym. I took out a membership. I...,Company chooses not to provide a public response,Thrive National Corporation,MS,Consent provided,Web,Closed with explanation,Yes,0
2087336,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,"I had XXXX XX/XX/XXXX and the XXXX, XXXX, XXXX...",Company believes it acted appropriately as aut...,Monterey Financial Services LLC,ID,Consent provided,Web,Closed with explanation,Yes,1
1951383,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,XXXX XXXX tells me I owe {$620.00} over 8 bill...,Company believes it acted appropriately as aut...,"Amsher Collection Services, Inc.",NC,Consent provided,Web,Closed with explanation,Yes,0
2220703,Debt collection,I do not know,Disclosure verification of debt,Not given enough info to verify debt,Have been turned over to collection on a debt ...,Company believes it acted appropriately as aut...,CONRAD CREDIT CORPORATION,CO,Consent provided,Web,Closed with explanation,Yes,0


In [5]:
unique_df = pd.DataFrame()
unique_df['columns'] = complaints_df.columns
unique_df['valid_count'] = complaints_df.count(axis=0).reset_index()[0]
unique_df['unique_count'] = complaints_df.nunique().reset_index()[0]
unique_df

,columns,valid_count,unique_count
0,product,19432,2
1,sub_product,19432,10
2,issue,19432,9
3,sub_issue,19432,37
4,consumer_complaint_narrative,19432,19146
5,company_public_response,19432,10
6,company,19432,1425
7,state,19432,57
8,consumer_consent_provided,19432,1
9,submitted_via,19432,1


In [6]:
target = pd.DataFrame(complaints_df.value_counts('consumer_disputed')).reset_index()
target.columns = ['consumer_disputed','count']
alt.Chart(target).mark_bar().encode(
    x=alt.X('consumer_disputed:O',title = 'Consumer Disputed'),
    y=alt.Y('count:Q',title = 'Count'),
    color='consumer_disputed:O',
)

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [7]:
X_train, y_train = train_df.drop(columns= ['consumer_disputed']), train_df['consumer_disputed']
X_test, y_test = test_df.drop(columns= ['consumer_disputed']), train_df['consumer_disputed']

In [8]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate

categorical_features = ['product',
                        'sub_product',
                        'issue',
                        'sub_issue',
                        'company_public_response', 
                        'company',
                        'state',
                        'consumer_consent_provided',
                        'consumer_consent_provided',
                        'submitted_via',
                        'company_response_to_consumer',
                        'timely_response']
drop_features = ['consumer_consent_provided','submitted_via']


text_feature = 'consumer_complaint_narrative'

preprocessor = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore',
                   drop='if_binary'), categorical_features),
    (CountVectorizer(stop_words='english', max_features = 1000), text_feature),
    ('drop', drop_features))
scoring_metrics = ['accuracy','recall','precision','f1']

cross_val_results = {}


In [9]:

from sklearn.dummy import DummyClassifier
pipe_dc = make_pipeline(preprocessor, DummyClassifier())
pipe_dc.fit(X_train, y_train)
cross_val_results['dummy'] = pd.DataFrame(cross_validate(
    pipe_dc, X_train, y_train,scoring=scoring_metrics)).agg(['mean']).round(3).T

In [10]:
cross_val_results['dummy']

,mean
fit_time,1.503
score_time,0.348
test_accuracy,0.779
test_recall,0.000
test_precision,0.000
test_f1,0.000


In [11]:
from sklearn.linear_model import LogisticRegression
pipe_lr = make_pipeline(preprocessor, LogisticRegression(max_iter=1000, class_weight='balanced'))
cross_val_results['logreg'] = pd.DataFrame(cross_validate(
    pipe_lr, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['logreg']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded 

,mean
fit_time,11.344
score_time,1.869
test_accuracy,0.634
test_recall,0.514
test_precision,0.305
test_f1,0.383


In [12]:
from sklearn.naive_bayes import BernoulliNB
pipe_nb = make_pipeline(preprocessor, BernoulliNB(alpha = 0.1))
cross_val_results['bayes'] = pd.DataFrame(cross_validate(
    pipe_nb, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['bayes']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/lu

,mean
fit_time,5.063
score_time,1.361
test_accuracy,0.696
test_recall,0.355
test_precision,0.327
test_f1,0.341


In [13]:
from sklearn.svm import SVC
pipe_svc = make_pipeline(preprocessor, SVC(class_weight='balanced'))
cross_val_results['svc'] = pd.DataFrame(cross_validate(
    pipe_svc, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['svc']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/

,mean
fit_time,215.497
score_time,44.811
test_accuracy,0.668
test_recall,0.466
test_precision,0.325
test_f1,0.383


In [14]:
from sklearn.ensemble import RandomForestClassifier
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(class_weight='balanced'))
cross_val_results['random forest'] = pd.DataFrame(cross_validate(
    pipe_rf, X_train, y_train, n_jobs=-1, scoring=scoring_metrics)).agg(['mean']).round(3).T
cross_val_results['random forest']

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/lu

,mean
fit_time,43.299
score_time,2.775
test_accuracy,0.783
test_recall,0.052
test_precision,0.624
test_f1,0.096


In [15]:
res = pd.concat(cross_val_results, axis=1)

In [16]:
res.columns = res.columns.droplevel(1)

In [17]:
res = res.reset_index()

In [18]:
res[2:]

,index,dummy,logreg,bayes,svc,random forest
2,test_accuracy,0.779,0.634,0.696,0.668,0.783
3,test_recall,0.000,0.514,0.355,0.466,0.052
4,test_precision,0.000,0.305,0.327,0.325,0.624
5,test_f1,0.000,0.383,0.341,0.383,0.096


In [19]:
source = res[2:].melt(id_vars=['index'])

In [20]:
source.columns = ['Metric','Model','Score']
source['Metric'] = source['Metric'].str.replace('test_','')

In [21]:

alt.Chart(source).mark_bar().encode(
    x='Metric:O',
    y='Score:Q',
    color='Metric:N',
    column='Model:N'
).save('test.png')

/Users/luke/opt/miniconda3/envs/MDS/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


In [23]:
models = {'Dummy Classifier':pipe_dc, 
          'Logistic Regression':pipe_lr, 
          'Naive Bayes Classifier':pipe_nb,
          'Support Vector Classifier':pipe_svc,
          'Random Forest Classifier':pipe_rf}
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
for model in models:
    models[model].fit(X_train,y_train)
    scores = [accuracy_score(models[model].predict(X_test), y_test),
              recall_score(models[model].predict(X_test), y_test),
              precision_score(models[model].predict(X_test), y_test),
              f1_score(models[model].predict(X_test), y_test)]
    

ValueError: Found input variables with inconsistent numbers of samples: [3887, 15545]